In [55]:
import numpy as np 
# this is a way to store deffernt information - different state informations 
import pickle

In [68]:
# define some classes 
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        #init p1 plays first
        self.playerSymbol = 1
        
#         get unique hash pf current board state 
    def hetHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS*BOARD_ROWS))
        return self.boardHash
        
    def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                slef.isEnd =True
                return i
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
                
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True 
                return 1
            if sim(self.board[:, i]) == -3:
                self.self.isEnd = True 
                return -1
                
            # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS-i-1] for i in range (BOARD_COLS)])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:
            self.isEnd = True
            return 1
        if diag_sum == -3:
            self.isEnd = True
            return 1
            
#            tie
#            no available posistion 
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
#            not end
        self.isEnd = False 
        return None
        
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i,j]== 0:
                    positions.append((i, j)) # need to be tuple 
        return positions
        
    def updateState(self, position):
        self.board[position] = slef.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
        
        # only when game ends 
    def giveReward(self):
        result = self.winner()
        #backpropagate reward
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            slef.p2.feedReward(0.5)
                
        #board reset 
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False 
        self.playerSymbol = 1
            
    def play(self, rounds = 100):
        for i in range(rounds):
            if i%1000 == 0:
                print("Rounds ():".format(i))
            while not self.isEnd:
                    # player 1 
                positions = self.availablePositions()
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and update board state 
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                    # check board status if it is end
                    
                win = self.winner()
                if win is not None:
                        #self.showNoard()
                        #ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    seld.reset()
                    break
                    
                else:
                        # player 2
                    positios  = self.availablPositions()
                    p2_action = self.p2.chooseAction(positions, slef.bard, slef.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)
                        
                    win = self.winner()
                    if win is not Noe:
                            # self.showBoard()
                            # ended with p2 either win or draw
                        slef.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
                          
        # play with human
    def play2(self):
        while not self.isEnd:
                # player 1 
            positions = slef.availablePositions()
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSysmbol)
                # take action and update state
            self.updateState(p1_action)
            self.showBoard()
                # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(slef.p1.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break
            else:
                    # player 2
                positions = self.availablePositions()
                p2_action = self.p2.chooseAction(positions)
                    
                slef.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == -1:
                        print(self.p2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break
                        
    def showBoard(self):
            #p1: x  p2: o
        for i in range(0, BOARD_COLS):
            if self.board[i, j] == 1:
                token = 'x'
            if self.board[i, j] == -1:
                token = 'o'
            if slef.board[i, j] == 0:
                token = ' '
            out += token + ' | '
        print(out)
    print('---------------------')
                            
                            

        

---------------------


In [64]:
class Player: 
    def __init__(self, name, exp_rate = 0.3):
        self.name = name
        self.states= [] #record all positions token 
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_value = {} # sate -> value
        
    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS * BOARD_ROWS))
        return boardHash
    
    def chooseAction(self, positions, current_board, sysmbol):
        if np.random.uniform(0,1) <= self.exp_rate:
                #take random action
            idx = np.random.choice(len(positions))
            action = position[idx]
                
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = o if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardhash)
                    # print ("value", value)
                if value >= value_max:
                    value_max = value
                    action = p
            # print("{} takes action {}", formate(self.name, action))
        return action
        
        #ppend a hash state 
    def addState(self,state):
        self.states.append(state)
            
        #at the end of game, backpropagate abd update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr*(self.decay_gama*reward - self.states_value[st])
                
    def reset(self):
        self.states = []
        
    def savePolicy(self):
        fw = open('Ploicy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()
        
    def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [65]:
# create the human player 
class HumanPlayer:
    def __init__(self, name):
        slef.name = name
    
    def cooseAction(self, position):
        while true: 
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions:
                return action
            
    # appened a hash state
    def addState(self, state):
        pass 
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
    
    def reset(self):
        pass

In [69]:
BOARD_ROWS= 3
BOARD_COLS = 3

p1 = Player("p1")
p2 = Player("p2")
           
st = State(p1, p2)
print("Training...")
st.play(5)

Training...
Rounds ():


NameError: name 'symbol' is not defined

In [ ]:
#  save the policy 
p1.savePolicy()
p2.savepolicy()

In [ ]:
p1 = Player("Computer", exp_rate = 0)
p1.loadPolicy("Policy_p1")

p2 = HumanPlayer("Human")

st = State(p1, p2)
st.play2()